<a href="https://colab.research.google.com/github/AviaraOdyssey/ScienceFair-FieldDataCNN/blob/main/FieldCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import numpy as py
import cv2
import os


In [ ]:
tf.random.set_seed(0)

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/gdrive', force_remount=True)



In [ ]:
# Location of Zip File
drive_path = '/gdrive/MyDrive/DataField/data.zip'
local_path = '/content'

# Copy the zip file and move it up one level (AKA out of the drive folder)
!cp '{drive_path}' .

# Navigate to the copied file and unzip it quietly
os.chdir(local_path)
!unzip -q 'data.zip'

In [ ]:
# Create data generators
# File path = path to test folders respectively
# Use a target size of 200x200 px for each image 
# Batch size = total number of images in the test set 
# Ensure class_mode is binary
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
                                                   '/content/FieldData/testing',
                                                     target_size=(200, 200),
                                                     batch_size=52,
                                                     class_mode='binary',
                                                    
                                                   )

In [ ]:
# Create data generators
# File path = path to train folders respectively
# Use a target size of 200x200 px for each image 
# Batch size = total number of images in the train set 
# Ensure class_mode is binary
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
                                                    '/content/FieldData/training',
                                                     target_size=(200, 200),
                                                     batch_size=77,
                                                     class_mode='binary',
                                                     subset = 'training'
                                                     )

# Iterate through each generator to create the data sets with the train/test splits
X_train, y_train = next(train_generator)
X_test, y_test = next(test_generator)

# Check which class is which
train_generator.class_indices

In [ ]:
#To verify that the dataset looks correct, plot the first 25 images from the training set and display the class name below each image
class_names = ['healthy', 'nematodes']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i])
plt.show()
y_train

In [ ]:
#Building the CNN Model
#Initialize model through Sequential module
#Implementation of Convolutional Neural network through Conv2D and MaxPooling2D Layers 
#Tensors of shape (image_height, image_width, color_channels)
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [ ]:
#Flatten Step: Convert MaxPooling2D into single continuous linear vector for input node of fully connected layers
#Dense Layer: Utilized to make a hidden layer where every neuron is connected to every neuron in the next layer (to perform classification).
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

In [ ]:
model.summary()

In [ ]:
#Method to stop overfitting of the model
early_stop = EarlyStopping(monitor='val_loss', patience=2)

In [ ]:
#Compile and train the CNN model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=20, 
                    validation_data=(X_test, y_test), callbacks=[early_stop])

In [ ]:
#Overall test accuracy of model
evaluation = model.evaluate(X_test, y_test)
print(f'Test Accuracy : {evaluation[1] * 100:.2f}%')

In [ ]:
#Evaluate Model
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

In [ ]:
#Confusion Matrix
y_pred = model.predict(X_test)
y_pred = py.argmax(y_pred, axis=1)
cm = confusion_matrix(y_test, y_pred)

In [ ]:
#Confusion Matrix
labels=['healthy','nematodes']
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=labels)


# NOTE: Fill all variables here with default values of the plot_confusion_matrix
fig, ax = plt.subplots(figsize=(2, 2))
disp = disp.plot(xticks_rotation='vertical', ax=ax,cmap='summer')

plt.show()